### Carregar dados disco local

In [ ]:
import pandas as pd

enfermagem = pd.read_csv('./datasets/enfermagem/all.csv')
enfermagem = enfermagem[enfermagem.status_aprendiz == 'Concluído']
enfermagem = enfermagem[enfermagem.titulo_objeto.str.startswith("Quiz")]

demais_cursos = pd.read_csv('./datasets/demais/demais_cursos.csv')
demais_cursos = demais_cursos[demais_cursos.status_aprendiz == 'Concluído']
demais_cursos = demais_cursos[demais_cursos.titulo_objeto.str.startswith("Quiz")]

### Carregando dados Google Drive (Somente se estiver usando o Google Colab)

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

enfermagem = pd.read_csv('/content/drive/MyDrive/datasets/proz/nivelamento_enfermagem.csv')
enfermagem = enfermagem.sort_values(by=['titulo_objeto'])


demais_cursos = pd.read_csv('/content/drive/MyDrive/datasets/proz/demais_cursos.csv')
demais_cursos = demais_cursos.sort_values(by=['titulo_objeto'])

### Função para geração dos Histogramas

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go


def plot_quiz_hist(quiz_data, n_rows, n_cols):
   
    quiz_range = len(quiz_data[quiz_data.titulo_objeto.str.endswith("D")].titulo_objeto.unique())

    fig = make_subplots(
        rows=n_rows,
        cols=n_cols,
        subplot_titles=([(lambda x: f'Quiz {x} / {x} D'.format(x))(x)
                        for x in range(1, quiz_range + 1)])
    )

    subplots_rows = 1
    subplots_cols = 1

    for i in range(1, quiz_range + 1):
        x0 = quiz_data[quiz_data.titulo_objeto ==
                       f'Quiz {i}'.format(i)].pontuacao

        x1 = quiz_data[quiz_data.titulo_objeto ==
                       f'Quiz {i} D'.format(i)].pontuacao

        y0 = quiz_data[quiz_data.titulo_objeto ==
                       f'Quiz {i}'.format(i)]

        y1 = quiz_data[quiz_data.titulo_objeto ==
                       f'Quiz {i} D'.format(i)]
        q_number = i

        fig.add_trace(
            go.Histogram(
                x=x0,
                y=y0,
                name=f'Quiz {q_number}'.format(q_number),
                nbinsx=10,
                marker_color='#FF7F00'
            ),
            row=subplots_rows, col=subplots_cols
        )

        fig.add_trace(
            go.Histogram(
                x=x1,
                y=y1,
                name=f'Quiz {q_number} D'.format(q_number),
                nbinsx=10,
                marker_color='#593493',
                opacity=0.55
            ),
            row=subplots_rows, col=subplots_cols
        )

        if subplots_cols == n_cols:
            subplots_cols = 1
            subplots_rows += 1
        else:
            subplots_cols += 1

    fig.update_layout(barmode='overlay', height=1000, width=1250,
                      title_text="Quiz", showlegend=False)
    fig.update_traces(histnorm="density")
    fig.update_xaxes(range=[0, 100], tick0=0, dtick=20)
    fig.update_yaxes(range=[0, 1], tick0=0, dtick=0.2)
    fig.show()

### Função para geração dos Bar Plots

In [ ]:

def create_quiz_metrics_dataset(quiz_data):

        quiz_range = len(quiz_data[quiz_data.titulo_objeto.str.endswith("D")].titulo_objeto.unique())

        quiz_list = list()
        for quiz_number in range(1, quiz_range + 1):
                quiz = {
                        "quiz": f'Quiz {quiz_number}'.format(quiz_number),
                        "mean": quiz_data[quiz_data.titulo_objeto == f'Quiz {quiz_number}'.format(quiz_number)].pontuacao.describe().mean().round(2) ,
                        "median": quiz_data[quiz_data.titulo_objeto == f'Quiz {quiz_number}'.format(quiz_number)].pontuacao.describe().median().round(2) 
                }

                quiz_list.append(quiz)

                quiz_d = {
                        "quiz": f'Quiz {quiz_number} D'.format(quiz_number),
                        "mean": quiz_data[quiz_data.titulo_objeto == f'Quiz {quiz_number} D'.format(quiz_number)].pontuacao.describe().mean().round(2) ,
                        "median": quiz_data[quiz_data.titulo_objeto == f'Quiz {quiz_number} D'.format(quiz_number)].pontuacao.describe().median().round(2) 
                }
                
                quiz_list.append(quiz_d)

        quiz_metrics = pd.DataFrame(quiz_list)
        return quiz_metrics

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

def plot_quiz_bar(quiz_data, metric, n_rows, n_cols):

    quiz_range = len(quiz_data[quiz_data.quiz.str.endswith("D")].quiz.unique())

    fig = make_subplots(
        rows=n_rows,
        cols=n_cols,
        subplot_titles=([(lambda x: f'Quiz {x} / {x} D'.format(x))(x)
                         for x in range(1, quiz_range + 1)])
    )

    subplots_rows = 1
    subplots_cols = 1

    for i in range(1, quiz_range + 1):

        y0 = quiz_data[quiz_data.quiz ==
                       f'Quiz {i}'.format(i)][metric]
        y1 = quiz_data[quiz_data.quiz ==
                       f'Quiz {i} D'.format(i)][metric]
        q_number = i

        fig.add_trace(
            go.Bar(
                x=['Quiz'],
                y=y0,
                name=f'Quiz {q_number} {metric}'.format(q_number, metric),
                marker_color='#FF7F00'
            ),
            row=subplots_rows, col=subplots_cols
        )

        fig.add_trace(
            go.Bar(
                x=['Quiz D'],
                y=y1,
                name=f'quiz {q_number} D {metric}'.format(q_number, metric),
                marker_color='#593493'
            ),
            row=subplots_rows, col=subplots_cols
        )

        if subplots_cols == n_cols:
            subplots_cols = 1
            subplots_rows += 1
        else:
            subplots_cols += 1

    fig.update_layout(height=1000, width=1250,
                      title_text="Quiz", showlegend=False)
    fig.update_yaxes(range=[0, 100], tick0=0, dtick=20)
    fig.show()

### Função para geração dos BoxPlots

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots


def plot_quiz_box(quiz_data, n_rows, n_cols):

    quiz_range = len(quiz_data[quiz_data.titulo_objeto.str.endswith("D")].titulo_objeto.unique())

    fig = make_subplots(
        rows=n_rows,
        cols=n_cols,
        subplot_titles=([(lambda x: f'Quiz {x}'.format(x))(x)
                        for x in range(1, quiz_range + 1)])
    )

    subplots_rows = 1
    subplots_cols = 1

    for i in range(1, quiz_range + 1):

        y0 = quiz_data[quiz_data.titulo_objeto ==
                             f'Quiz {i}'.format(i)].pontuacao
        y1 = quiz_data[quiz_data.titulo_objeto ==
                             f'Quiz {i} D'.format(i)].pontuacao

        q_number = i

        fig.add_trace(
                  go.Box(
                      y=y0,
                      name=f'Quiz {q_number}'.format(q_number),
                      marker_color='#FF7F00'
                  ),
                  row=subplots_rows, col=subplots_cols
              )

        fig.add_trace(
                  go.Box(
                      y=y1,
                      name=f'Quiz {q_number} D'.format(q_number),
                      marker_color='#593493'
                  ),
                  row=subplots_rows, col=subplots_cols
              )

        if subplots_cols == n_cols:
            subplots_cols = 1
            subplots_rows += 1
        else:
            subplots_cols += 1


    fig.update_layout(title_text="Quiz", showlegend=False, 
                      height=1000, width=1250,)
    fig.update_traces(boxmean=True)
    fig.update_traces(orientation='v')
    fig.show()


def plot_quiz_box_total(quiz_data):
    y0 = quiz_data[quiz_data.titulo_objeto.str.endswith("D")].pontuacao
    y1 = quiz_data[~quiz_data.titulo_objeto.str.endswith("D")].pontuacao
    fig = go.Figure()
    fig.add_trace(go.Box(y=y0,name='Quiz', marker_color='#FF7F00'))
    fig.add_trace(go.Box(y=y1,name='Quiz D',marker_color='#593493'))
    fig.update_traces(boxmean=True)
    fig.update_traces(orientation='v')
    fig.update_layout(title_text="Quiz Demais Cursos", height=400, width=600)
    fig.show()

### Visualizando Histogramas

In [ ]:
plot_quiz_hist(quiz_data=enfermagem, n_rows=5, n_cols=3)

In [ ]:
plot_quiz_hist(quiz_data=demais_cursos, n_rows=5, n_cols=3)

### Visualizando BarPlots

In [ ]:
quiz_metrics=create_quiz_metrics_dataset(enfermagem)
plot_quiz_bar(quiz_data=quiz_metrics,metric='mean', n_rows=5, n_cols=3)

### Visualizando BoxPlots

In [ ]:
plot_quiz_box(quiz_data=enfermagem, n_rows=5, n_cols=3)

In [ ]:
plot_quiz_box(quiz_data=demais_cursos, n_rows=5, n_cols=3)

In [ ]:
plot_quiz_box_total(enfermagem)